# Notebook 1: Preparación de Datos para RAG

Este notebook cubre el primer paso del sistema RAG:
- Cargar archivos Excel de presupuestos
- Procesar y normalizar partidas
- Preparar datos para indexación semántica

**Flujo**: Excel → JSON → Dataset Normalizado

## 1. Importar Librerías

In [35]:
import pandas as pd
import json
import re
import unicodedata
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any

## 2. Configuración de Rutas

In [36]:
BASE_DIR = Path.cwd()
DATASET_FILE = BASE_DIR / "partidas_embedding_ready.json"
EXCEL_FILE = BASE_DIR / "/Users/alexmartin/Documents/KC_projects/propaher/presupuesto_vivienda.xlsx"

print(f"Directorio base: {BASE_DIR}")
print(f"Archivo Excel: {EXCEL_FILE.name if EXCEL_FILE.exists() else 'No encontrado'}")

Directorio base: /Users/alexmartin/Documents/KC_projects/propaher
Archivo Excel: presupuesto_vivienda.xlsx


## 3. Funciones de Procesamiento

In [37]:
def to_float(value):
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return None
    if isinstance(value, (int, float)):
        return float(value)
    if isinstance(value, str):
        value = value.replace("€", "").strip()
        value = value.replace(".", "").replace(",", ".")
        try:
            return float(value)
        except:
            return None
    return None

def is_capitulo(text):
    if not isinstance(text, str):
        return False
    text = text.strip()
    return (
        text.isupper() and
        len(text) < 80 and
        not any(char.isdigit() for char in text)
    )

def is_noise(text):
    if not isinstance(text, str):
        return True
    text = text.lower()
    return any(x in text for x in [
        "nota:", "presupuesto", "valoramos", "nan", "modificación"
    ])

def find_header_row(df):
    keywords = ["concepto", "descripción", "unidad", "cantidad", "precio", "importe"]
    for i, row in df.iterrows():
        text = " ".join(str(x).lower() for x in row if pd.notna(x))
        if any(k in text for k in keywords):
            return i
    return None

## 4. Funciones de Normalización

In [38]:
def separar_concepto(concepto):
    concepto = str(concepto).strip()
    if not concepto or concepto.lower() == "nan":
        return "", ""
    
    separadores = [r"formada per", r"formado por", r"compuesta por",
                   r"inclou:", r"incluye:", r"segons", r"según"]
    
    for sep in separadores:
        match = re.search(sep, concepto, re.IGNORECASE)
        if match:
            return concepto[:match.start()].strip(), concepto[match.start():].strip()
    
    if len(concepto) > 150:
        coma = concepto.find(',', 50)
        if 0 < coma < 120:
            return concepto[:coma].strip(), concepto[coma+1:].strip()
        return concepto[:150] + "...", concepto
    
    return concepto, ""

def normalizar_unidad(p):
    unidad = str(p.get("unidad", "")).lower().strip()
    concepto = str(p.get("concepto", "")).lower()
    
    mapeo = {
        "ud": "ud", "ut": "ud", "ut.": "ud", "u": "ud",
        "m": "m", "ml": "m", "mts": "m", "mts.": "m",
        "m2": "m2", "m²": "m2",
        "pa": "pa", "p.a.": "pa"
    }
    
    for k, v in mapeo.items():
        if k in unidad:
            return v
    
    if "per habitatge" in concepto or "vivienda" in concepto:
        return "vivienda"
    if "centralització" in concepto:
        return "centralizacion"
    
    return "ud"

def normalizar_concepto(concepto):
    if not concepto:
        return ""
    
    texto = str(concepto).lower().strip()
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(c for c in texto if unicodedata.category(c) != 'Mn')
    
    sinonimos = {
        "habitatge": "vivienda", "safata": "bandeja", "quadre": "cuadro",
        "caixa": "caja", "linia": "linea", "comptador": "contador",
        "subministrament": "suministro", "col·locacio": "colocacion",
        "collocacio": "colocacion", "instal·lacio": "instalacion",
        "installacio": "instalacion"
    }
    
    for cat, cast in sinonimos.items():
        texto = texto.replace(cat, cast)
    
    vacias = ["de", "del", "la", "el", "per", "para", "amb", "con", "i", "y", "a", "en"]
    palabras = [p for p in texto.split() if p not in vacias and len(p) > 1]
    
    return re.sub(r'[^\w\s]', '', " ".join(palabras)).strip()

def clasificar_tipo(p):
    concepto = str(p.get("concepto", "")).strip()
    cantidad = p.get("cantidad")
    precio = p.get("precio_unitario")
    importe = p.get("importe")
    
    if concepto.startswith(".-") or (concepto.startswith("-") and len(concepto) > 1):
        return "subdetalle"
    
    if concepto.isupper() and len(concepto) < 80:
        if cantidad is None and importe is None:
            return "capitulo"
    
    if precio and precio > 0:
        if cantidad is None and importe is None:
            return "resumen"
    
    if cantidad is not None and precio is not None and importe is not None:
        if cantidad > 0 and precio > 0 and importe > 0:
            return "partida"
    
    return "otro"

## 5. Cargar y Procesar Excel

In [39]:
def procesar_excel(ruta_excel):
    print(f"Procesando: {ruta_excel.name}")
    
    df_raw = pd.read_excel(ruta_excel, header=None)
    
    header_row = find_header_row(df_raw)
    if header_row is None:
        print("No se encontró fila de cabecera")
        return []
    
    df = pd.read_excel(ruta_excel, header=header_row)
    df.columns = [str(c).lower().strip() for c in df.columns]
    df = df.dropna(how="all")
    
    print(f"Filas encontradas: {len(df)}")
    
    current_capitulo = None
    partidas = []
    
    for _, row in df.iterrows():
        concepto = str(row.get("concepto", "")).strip()
        
        if concepto == "" or is_noise(concepto):
            continue
        
        if is_capitulo(concepto):
            current_capitulo = concepto
            continue
        
        cantidad = to_float(row.get("cantidad"))
        precio = to_float(row.get("precio"))
        importe = to_float(row.get("importe"))
        
        if cantidad is None and importe is None:
            continue
        
        p = {
            "capitulo": current_capitulo,
            "concepto": concepto,
            "unidad": str(row.get("med", row.get("unidad", ""))).strip(),
            "cantidad": cantidad,
            "precio_unitario": precio,
            "importe": importe
        }
        
        p["tipo"] = clasificar_tipo(p)
        base, tecnica = separar_concepto(p.get("concepto", ""))
        p["concepto_base"] = base
        p["descripcion_tecnica"] = tecnica
        p["unidad_normalizada"] = normalizar_unidad(p)
        p["concepto_normalizado"] = normalizar_concepto(base)
        
        partidas.append(p)
    
    partidas_limpias = [
        {
            "capitulo": p.get("capitulo"),
            "concepto_base": p.get("concepto_base"),
            "concepto_normalizado": p.get("concepto_normalizado"),
            "descripcion_tecnica": p.get("descripcion_tecnica"),
            "unidad": p.get("unidad_normalizada"),
            "cantidad": p.get("cantidad"),
            "precio_unitario": p.get("precio_unitario"),
            "importe": p.get("importe"),
            "origen": ruta_excel.stem,
            "archivo": ruta_excel.name
        }
        for p in partidas
        if p.get("tipo") == "partida" and
           p.get("cantidad") and p.get("cantidad") > 0 and
           p.get("precio_unitario") and p.get("precio_unitario") > 0 and
           p.get("importe") and p.get("importe") > 0
    ]
    
    print(f"Extraídas {len(partidas_limpias)} partidas válidas de {len(partidas)} totales")
    
    return partidas_limpias

if EXCEL_FILE.exists():
    partidas = procesar_excel(EXCEL_FILE)
else:
    print("No se encontró el archivo Excel. Especifica la ruta correcta.")
    partidas = []

Procesando: presupuesto_vivienda.xlsx
Filas encontradas: 31
Extraídas 24 partidas válidas de 24 totales


## 6. Explorar Partidas Procesadas

In [40]:
if partidas:
    df_partidas = pd.DataFrame(partidas)
    
    print(f"Total partidas: {len(df_partidas)}")
    display(df_partidas.head())
    
    print("\nEstadísticas de precios:")
    display(df_partidas['precio_unitario'].describe())
    
    print("\nDistribución por capítulo:")
    display(df_partidas['capitulo'].value_counts())
else:
    print("No hay partidas para mostrar")

Total partidas: 24


,capitulo,concepto_base,concepto_normalizado,descripcion_tecnica,unidad,cantidad,precio_unitario,importe,origen,archivo
0,INSTALACIÓN ELÉCTRICA,"Cuadro eléctrico vivienda 4 elementos, incluye...",cuadro electrico vivienda elementos incluye ic...,,ud,1.0,245.50,245.5,presupuesto_vivienda,presupuesto_vivienda.xlsx
1,INSTALACIÓN ELÉCTRICA,Bandeja portacables de acero galvanizado 100x6...,bandeja portacables acero galvanizado 100x60mm...,,m,35.0,12.80,448.0,presupuesto_vivienda,presupuesto_vivienda.xlsx
2,INSTALACIÓN ELÉCTRICA,"Tubo corrugado M20 doble capa para empotrar, i...",tubo corrugado m20 doble capa empotrar incluye...,,m,180.0,0.85,153.0,presupuesto_vivienda,presupuesto_vivienda.xlsx
3,INSTALACIÓN ELÉCTRICA,Cable RZ1-K 3x2.5mm2 libre de halógenos para c...,cable rz1k 3x25mm2 libre halogenos circuitos a...,,m,220.0,1.35,297.0,presupuesto_vivienda,presupuesto_vivienda.xlsx
4,INSTALACIÓN ELÉCTRICA,Cable RZ1-K 3x4mm2 libre de halógenos para cir...,cable rz1k 3x4mm2 libre halogenos circuitos fu...,,m,150.0,2.15,322.5,presupuesto_vivienda,presupuesto_vivienda.xlsx



Estadísticas de precios:


count     24.000000
mean      52.741667
std      145.552681
min        0.650000
25%        2.300000
50%        4.525000
75%       12.575000
max      685.000000
Name: precio_unitario, dtype: float64


Distribución por capítulo:


capitulo
INSTALACIÓN ELÉCTRICA                8
INSTALACIÓN DE FONTANERÍA            8
INSTALACIÓN DE TELECOMUNICACIONES    5
INSTALACIÓN DE CLIMATIZACIÓN         3
Name: count, dtype: int64

## 7. Guardar Dataset Preparado

In [41]:
if partidas:
    dataset = {
        "metadata": {
            "version": "1.0",
            "fecha_creacion": datetime.now().isoformat(),
            "total_partidas": len(partidas),
            "fuente": EXCEL_FILE.name
        },
        "partidas": partidas
    }
    
    with open(DATASET_FILE, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)
    
    print(f"Dataset guardado en: {DATASET_FILE.name}")
    print(f"Tamaño: {DATASET_FILE.stat().st_size / 1024:.1f} KB")
else:
    print("No hay partidas para guardar")

Dataset guardado en: partidas_embedding_ready.json
Tamaño: 10.7 KB
